<a href="https://colab.research.google.com/github/Gobihanath/DTect/blob/main/DTect_RestNet50.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import PIL
from tensorflow import keras
from keras import layers
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator,img_to_array,load_img
from keras import layers, models, callbacks
from keras.applications import ResNet50


input_shape = (256,256 , 3)

In [2]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [3]:
Image_Width = 256
Image_Height = 256
Image_Size = (Image_Width, Image_Height)
Image_Channel = 3
batch_size=128

In [4]:
data_set = tf.keras.preprocessing.image_dataset_from_directory(
    "/content/drive/MyDrive/Dataset/Dtect | Dataset | New",
    seed=123,         #If you want to ensure that the shuffling of your dataset is the same across different runs, you can keep the seed value constant. If you don't care about reproducibility, you can omit the seed parameter or use a different value for each run.
    shuffle=True,
    image_size=Image_Size,
    batch_size=batch_size,
    labels="inferred",
    label_mode="categorical",
    class_names=None,
    color_mode="rgb",
    subset=None,
    interpolation="bilinear",
    follow_links=False,
    crop_to_aspect_ratio=False
)

Found 5001 files belonging to 10 classes.


In [5]:
class_names = data_set.class_names
class_names

['Person 1',
 'Person 10',
 'Person 2',
 'Person 3',
 'Person 4',
 'Person 5',
 'Person 6',
 'Person 7',
 'Person 8',
 'Person 9']

In [ ]:
def get_dataset_partitions_tf(data_set, train_split=0.8,  val_split=0.2, shuffle=True, shuffle_size=10000):
    assert (train_split + val_split ) == 1

    ds_size = len(data_set)

    if shuffle:
        data_set = data_set.shuffle(shuffle_size, seed=123)

    train_size = int(train_split * ds_size)
    val_size = int(val_split * ds_size)

    train_ds = data_set.take(train_size)
    val_ds = data_set.skip(train_size).take(val_size)

    return train_ds, val_ds



train_ds, val_ds = get_dataset_partitions_tf(data_set)

print(len(train_ds))
print(len(val_ds))

32
8


In [ ]:
base_model= tf.keras.applications.resnet50.ResNet50(
                include_top=False,
                weights='imagenet',
                input_tensor=None,
                input_shape=input_shape,
            )

94765736/94765736 [==============================] - 0s 0us/step


In [ ]:
# Freeze the pre-trained layers
# don't train existing weights

for layer in base_model.layers:
    layer.trainable = False

In [ ]:
model = models.Sequential([
    base_model,
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(10, activation='softmax')
])

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 8, 8, 2048)        23587712  
                                                                 
 flatten_1 (Flatten)         (None, 131072)            0         
                                                                 
 dense_2 (Dense)             (None, 128)               16777344  
                                                                 
 dense_3 (Dense)             (None, 10)                1290      
                                                                 
Total params: 40366346 (153.99 MB)
Trainable params: 16778634 (64.01 MB)
Non-trainable params: 23587712 (89.98 MB)
_________________________________________________________________


In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
ACCURACY_THRESHOLD = 0.85

class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        accuracy = logs.get('accuracy')  # Use 'accuracy' instead of 'acc'
        if accuracy is not None and accuracy > ACCURACY_THRESHOLD:
            print("\nReached %2.2f%% accuracy, so stopping training!!" % (ACCURACY_THRESHOLD * 100))
            self.model.stop_training = True

# Instantiate a callback object
early_stopping = myCallback()

In [ ]:
history = model.fit(
    train_ds,
    batch_size=batch_size,
    validation_data=val_ds,
    validation_steps=8,
    epochs=20,
    callbacks=[early_stopping]
)


Epoch 1/20
32/32 [==============================] - 2540s 51s/step - loss: 4.2428 - accuracy: 0.7790 - val_loss: 0.2390 - val_accuracy: 0.9619
Epoch 2/20
32/32 [==============================] - ETA: 0s - loss: 0.0700 - accuracy: 0.9864 
Reached 85.00% accuracy, so stopping training!!
32/32 [==============================] - 1823s 51s/step - loss: 0.0700 - accuracy: 0.9864 - val_loss: 0.0050 - val_accuracy: 0.9971


In [ ]:
model.save('/content/drive/MyDrive/ML Trained Models/Dtect_Alternation_ResNet50_Model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [8]:
from tensorflow.keras.models import load_model

model = load_model('/content/drive/MyDrive/ML Trained Models/Dtect_Alternation_ResNet50_Model.h5')

In [9]:
import numpy as np
import tensorflow as tf

def predict(model, img, class_names, confidence_threshold=0.5):
    # Preprocess the image
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0)

    # Make predictions
    predictions = model.predict(img_array)

    # Extract predicted class and confidence
    predicted_class_index = np.argmax(predictions[0])
    confidence = predictions[0][predicted_class_index]
    predicted_class = class_names[predicted_class_index]

    # Check if confidence is high enough
    if confidence >= confidence_threshold:
        verification_status = "verified"
    else:
        verification_status = "not verified"

    return predicted_class, confidence, verification_status

In [11]:
# Load the trained model
model = tf.keras.models.load_model('/content/drive/MyDrive/ML Trained Models/Dtect_Alternation_ResNet50_Model.h5')

# Define class names
# Add your class names here

# Load the image you want to predict
img = tf.keras.preprocessing.image.load_img('/content/drive/MyDrive/T5.jpg', target_size=(256, 256))

# Predict the image
predicted_class, confidence, verification_status = predict(model, img, class_names)

# Print the results
print("Predicted class:", predicted_class)
print("Confidence:", confidence)
print("Verification status:", verification_status)

1/1 [==============================] - 1s 1s/step
Predicted class: Person 2
Confidence: 1.0
Verification status: verified


In [ ]:
def verify_signature(image_path):
    # Preprocess the image
    img = tf.keras.preprocessing.image.load_img(image_path, target_size=input_shape)
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0)  # Create batch axis
    img_array /= 255.  # Normalize

    # Classify the image
    predictions = model.predict(img_array)
    predicted_class = tf.argmax(predictions[0]).numpy()

    if predicted_class < 10:
        return f"Verified as class {predicted_class}"
    else:
        return "Not verified"

# Example usage
image_path = '/content/drive/MyDrive/Kb_test.jpg'
verification_result = verify_signature(image_path)
print(verification_result)

1/1 [==============================] - 0s 168ms/step
Verified as class 0


In [ ]:
from tensorflow.keras.preprocessing import image
def preprocess_image(image_path):
    img = image.load_img(image_path, target_size=(256,256))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    return img_array

# Function to make predictions on unseen images
def predict_image(model, image_path):
    img_array = preprocess_image(image_path)
    prediction = model.predict(img_array)
    predicted_class = np.argmax(prediction, axis=1)
    return predicted_class

# Test on unseen images
image_path = '/content/drive/MyDrive/orange3.jpg'
predicted_class = predict_image(model, image_path)
print('Predicted Class:', predicted_class)

1/1 [==============================] - 0s 232ms/step
Predicted Class: [0]
